In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
from pathlib import Path

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
# Load the data
file_path = "https://raw.githubusercontent.com/kwinterling/FinalProject/main/smoker_data/Resources/train_dataset.csv"
df = pd.read_csv(file_path)
#df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()


df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70,142,19.8,1,1.0,61,115,125,1,1
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0


In [3]:
df.dtypes

age                      int64
height(cm)               int64
weight(kg)               int64
waist(cm)              float64
eyesight(left)         float64
eyesight(right)        float64
hearing(left)            int64
hearing(right)           int64
systolic                 int64
relaxation               int64
fasting blood sugar      int64
Cholesterol              int64
triglyceride             int64
HDL                      int64
LDL                      int64
hemoglobin             float64
Urine protein            int64
serum creatinine       float64
AST                      int64
ALT                      int64
Gtp                      int64
dental caries            int64
smoking                  int64
dtype: object

In [4]:
# Drop the non-beneficial ID columns.
# new_df = df.drop(['waist(cm)', 'height(cm)', 'weight(kg)', 'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'relaxation', 'AST', 'age', 'systolic'], axis=1)
df = df[(df["age"] < 40) | (df["age"] > 45)]
new_df = df.drop(['eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'relaxation', 'AST', 'age', 'systolic'], axis=1)
# new_df = df
new_df.head()

,height(cm),weight(kg),waist(cm),fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,ALT,Gtp,smoking
0,170,85,97.0,97,239,153,70,142,19.8,1.0,115,125,1
1,175,110,110.0,88,211,128,71,114,15.9,1.1,25,30,0
4,165,60,81.0,100,179,200,47,92,14.9,1.2,28,15,0
5,160,50,78.0,114,177,74,98,64,13.9,1.0,23,70,1
11,155,50,72.0,83,135,35,59,69,12.5,0.9,12,11,0


In [5]:
X = new_df.drop(columns="smoking")
y = pd.DataFrame(new_df["smoking"])

In [6]:
X.describe()

,height(cm),weight(kg),waist(cm),fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,ALT,Gtp
count,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000,23371.000000
mean,165.054555,66.797955,83.108964,100.080014,195.551538,127.303924,56.354499,114.542467,14.854033,0.906769,28.280818,41.023405
std,9.586454,12.784210,8.842579,20.622269,37.319224,69.791427,14.292776,43.555451,1.371734,0.224229,33.486272,51.612537
min,130.000000,30.000000,51.000000,46.000000,77.000000,8.000000,4.000000,1.000000,6.300000,0.100000,1.000000,2.000000
25%,160.000000,60.000000,77.000000,89.000000,170.000000,76.000000,46.000000,90.000000,13.900000,0.800000,16.000000,18.000000
50%,165.000000,65.000000,83.000000,96.000000,193.000000,110.000000,55.000000,112.000000,15.000000,0.900000,22.000000,27.000000
75%,170.000000,75.000000,88.900000,105.000000,219.000000,160.000000,64.000000,136.000000,15.800000,1.000000,32.000000,44.000000
max,190.000000,135.000000,129.000000,398.000000,445.000000,466.000000,359.000000,1860.000000,20.900000,10.000000,2914.000000,999.000000


In [7]:
y['smoking'].value_counts()

0    14669
1     8702
Name: smoking, dtype: int64

In [8]:
from collections import Counter
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train.shape

(17528, 12)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(17528, 12)
(5843, 12)
(17528, 1)
(5843, 1)


In [10]:
Counter(y_train)

Counter({'smoking': 1})

In [11]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=200, max_depth=32, random_state=1)
brf

BalancedRandomForestClassifier(max_depth=32, n_estimators=200, random_state=1)

In [13]:
brf.fit(X_train_scaled, y_train)
y_pred = brf.predict(X_test_scaled)

<ipython-input-13-aff86d4fa7d9>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  brf.fit(X_train_scaled, y_train)


In [ ]:
rf = BalancedRandomForestClassifier()
parameters = {
    "n_estimators": [5, 20, 40, 80, 100, 200],
    "max_depth": [2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(X_train.values, y_train.values.ravel())

GridSearchCV(cv=5, estimator=BalancedRandomForestClassifier(),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'n_estimators': [5, 20, 40, 80, 100, 200]})

In [ ]:
print(cv.best_params_)

{'max_depth': 32, 'n_estimators': 80}


In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brf.predict(X_test_scaled)
y_pred

array([1, 0, 0, ..., 0, 1, 1])

In [15]:
balanced_accuracy_score(y_test, y_pred)

0.7709747706072159

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

array([[2643, 1096],
       [ 347, 1757]])

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      0.71      0.84      0.79      0.77      0.58      3739
          1       0.62      0.84      0.71      0.71      0.77      0.60      2104

avg / total       0.79      0.75      0.79      0.76      0.77      0.59      5843



In [18]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.12603620352392408, 'height(cm)'),
 (0.1243363515855268, 'hemoglobin'),
 (0.11377235068979014, 'Gtp'),
 (0.08856975396524183, 'triglyceride'),
 (0.07646794300985635, 'LDL'),
 (0.07635852730830733, 'Cholesterol'),
 (0.07435386789049413, 'ALT'),
 (0.07430404922719762, 'waist(cm)'),
 (0.0723612947989612, 'HDL'),
 (0.07151735655978154, 'fasting blood sugar'),
 (0.0516793621763137, 'serum creatinine'),
 (0.050242939264605296, 'weight(kg)')]

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

xgb = GradientBoostingClassifier()

xgb.fit(X_train_scaled, y_train.values.ravel())
y_pd = xgb.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pd)


0.6915019163945546